In [1]:
from graph_states import build_graph_state
import networkx as nx

n= 5
shots = 10000
G = nx.erdos_renyi_graph(n = n, p= 0.7)
state_circuit = build_graph_state(G)
state_circuit.draw(fold = -1)

┌───┐                  
q_0: ┤ H ├─■────────────────
     ├───┤ │                
q_1: ┤ H ├─┼──■──■──────────
     ├───┤ │  │  │          
q_2: ┤ H ├─■──┼──┼──■──■────
     ├───┤    │  │  │  │    
q_3: ┤ H ├────■──┼──■──┼──■─
     ├───┤       │     │  │ 
q_4: ┤ H ├───────■─────■──■─
     └───┘

In [2]:
from graph_states import graph_stabilizers, stabilizer_measurement_circuit, graph_witness, expectation_from_counts
from zne_entanglement_iqm import run_on_iqm_hardware
from qiskit.quantum_info import Pauli
edges = G.edges()
stabilizers = graph_stabilizers(n, edges)
meas_circs = [stabilizer_measurement_circuit(state_circuit, p) for p in stabilizers]

backend, result = run_on_iqm_hardware(
    meas_circs,
    server_url="https://resonance.meetiqm.com",
    quantum_computer="emerald",
    shots=shots,
    optimization_level=3,
    seed_transpiler=1,
)

expectations = []
for i, p in enumerate(stabilizers):
    counts = result.get_counts(i)
    expectations.append(expectation_from_counts(counts, p))

    

W = graph_witness(expectations)
print("Witness =", W)
if W < 0:
    print("Entangled")
else:
    print("Not Entangled")






/home/amaciejunes/Desktop/IQM/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Witness = 0.07140000000000013
Not Entangled


In [3]:
from qiskit.quantum_info import Statevector

sv = Statevector.from_instruction(state_circuit)

ideal_exps = []
for p in stabilizers:
    ideal_exps.append(float(sv.expectation_value(Pauli(p)).real))

ideal_W = graph_witness(ideal_exps)

print("Ideal expectations:")
for i, (p, e) in enumerate(zip(stabilizers, ideal_exps)):
    print(i, p, e)

print("Ideal W =", ideal_W)


Ideal expectations:
0 IIZIX 0.9999999999999992
1 ZZIXI 0.9999999999999992
2 ZZXIZ 0.9999999999999992
3 ZXZZI 0.9999999999999992
4 XZZZI 0.9999999999999992
Ideal W = -0.4999999999999982
